# Access GDD files in ArcGIS and HSDS

##### Get GDD file from HSDS to CSV

In [17]:
import h5pyd
import pandas as pd
from datetime import datetime, timedelta

# Get yesterday's date
date = (datetime.today() - timedelta(days=1)).strftime("%Y%m%d")

# Load the GDD file from HSDS
f = h5pyd.File(f"/home/usda1/mesonet/ldad-mesonet-{date}_gdd.nc4", 'r', 'http://4.151.248.251')

# Convert the data into a pandas dataframe
df = pd.DataFrame({
                  "stationId": [id.decode('utf-8') for id in f['stationId']][:], 
                  "obsTimeMin": [d.decode('utf-8') for d in f['obsTimeMin']][:],
                  "minTemp": [round(n,2) for n in f["minTemp"]][:],
                  "obsTimeMax": [d.decode('utf-8') for d in f['obsTimeMax']][:],
                  "maxTemp": [round(n,2) for n in f["maxTemp"]][:],
                  "avgTemp": [round(n,2) for n in f["avgTemp"]][:],
                  "gdd": [round(n,2) for n in f["gdd"]][:],
                  "latitude": [round(n,2) for n in f["latitude"]][:],
                  "longitude": [round(n,2) for n in f["longitude"]][:]
                  })

# Convert the dataframe into a CSV file
df.to_csv(f'ldad-mesonet-{date}_gdd.csv', escapechar='"', index=False)

##### Get GDD file from ArcGIS to CSV

In [18]:
import pandas as pd
import requests
from io import StringIO

import arcgis
from arcgis.gis import GIS
from datetime import datetime, timedelta

pdi = GIS(url="https://pdi-development.scinet.usda.gov/portal")

# Get yesterday's date
date = (datetime.today() - timedelta(days=1)).strftime("%Y%m%d")

csv_item = pdi.content.search(query=f'ldad-mesonet-{date}_gdd type:CSV')[0]
csv_url = f"https://pdi-development.scinet.usda.gov/portal/sharing/rest/content/items/{csv_item.id}/data"
response = requests.get(csv_url)
if response.status_code == 200:
    csv_data = StringIO(response.text)
    df = pd.read_csv(csv_data)
    df = df.drop(['fid', 'x', 'y'], axis=1)
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = round(df[col],2)
    df.to_csv(f'ldad-mesonet-{date}_gdd.csv', escapechar='"', index=False)
else:
    print(f"Failed to download file. Status code: {response.status_code}")